In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from path import Path as p

In [2]:
PATH = "../../../data/"
PATH2 = "../../../data/Flicker8k_Dataset/"
sz=224
bs = 64
n = 1000 # Number of dogs and cats to train model on
# imbalanced classes
CATDOG = 0 # Dummy class variables
NOTCATDOG = 1 # Not Cat or Not Dog

In [3]:
def get_names(filelist, suffix):
    return [suffix + f.name for f in filelist]

In [5]:
train_cats = get_names(p(PATH + "dogscats/train/cats/").files(), "dogscats/train/cats/")
train_dogs = get_names(p(PATH + "dogscats/train/dogs/").files(), "dogscats/train/dogs/")
valid_cats = get_names(p(PATH + "dogscats/valid/cats/").files(), "dogscats/valid/cats/")
valid_dogs = get_names(p(PATH + "dogscats/valid/dogs/").files(), "dogscats/valid/dogs/")
flickr = get_names(p(PATH2).files(), "Flicker8k_Dataset/")
print(len(train_cats))
print(len(train_dogs))
print(len(valid_cats))
print(len(valid_dogs))
print(len(flickr))

11500
11500
1000
1000
8091


## Part 1: Not Cats

In [6]:
def prep(train, valid):
    animals = list(np.random.choice(train, n, False)) + list(valid)
    notanimals = list(np.random.choice(flickr, n + len(valid), False))
    fn = animals + notanimals
    yy = np.array([CATDOG]*len(animals) + [NOTCATDOG]*len(notanimals))
    cs = list(set(yy))
    v_cat_dog_idx = range(n, n + len(valid))
    v_not_idx = range(len(animals) + n, len(fn))
    vi = list(v_cat_dog_idx) + list(v_not_idx)
    return (fn, yy, cs, vi)

# fn or fnames: file names
# yy or y: numpy array which contains target labels ordered by filenames.
# cs or classes: a list of all labels/classifications, [0, 1]
# vi or val_idxs: index of images to be used for validation.

In [6]:
fnames, y, classes, val_idxs = prep(train_cats, valid_cats)

In [7]:
# Let's set up our model using the pretrained Resnet34 Imagenet model
arch=resnet34
data = ImageClassifierData.from_names_and_array(PATH, fnames, y, classes, \
                                                val_idxs, bs=bs, \
                                                tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [8]:
# Use a learning rate of 0.01 and train for 5 epochs
lr = 0.01
epochs = 5
learn.fit(lr, epochs)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.143951   0.026907   0.992     
    1      0.062463   0.021382   0.993                                                                                 
    2      0.039411   0.017082   0.9945                                                                                
    3      0.02947    0.017213   0.995                                                                                 
    4      0.02241    0.014847   0.9945                                                                                



[array([0.01485]), 0.9945]

In [7]:
# Create our prediction function
def predict(learner, pred_files):
    orig_precompute = learner.precompute
    learner.precompute = False
    trn_tfms, val_tfms = tfms_from_model(arch, sz)
    ds = FilesIndexArrayDataset(list(pred_files), np.zeros(len(pred_files)), val_tfms, PATH)
    dl = DataLoader(ds)
    log_preds = learner.predict_dl(dl)
    preds = np.exp(log_preds)
    results = np.argmax(preds, axis=1)
    learner.precompute = orig_precompute
    return results

In [10]:
# Now try to predict on dogs
pred_dogs = predict(learn, valid_dogs)
print(sum(pred_dogs))

835


## Part 2: Not Dogs

In [8]:
fnames, y, classes, val_idxs = prep(list(np.random.choice(flickr, n, False)), valid_dogs)

In [11]:
# Let's set up our model using the pretrained Resnet34 Imagenet model
arch=resnet34
data = ImageClassifierData.from_names_and_array(PATH, fnames, y, classes, \
                                                val_idxs, bs=bs, \
                                                tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [12]:
# Use a learning rate of 0.01 and train for 5 epochs
lr = 0.01
epochs = 20
learn.fit(lr, epochs)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.877822   0.778913   0.433     
    1      0.806515   0.766466   0.441                                                                                 
    2      0.776795   0.690089   0.57                                                                                  
    3      0.746187   0.725484   0.511                                                                                 
    4      0.740291   0.724534   0.5245                                                                                
    5      0.725797   0.776966   0.4695                                                                                
    6      0.706959   0.758652   0.4995                                                                                
    7      0.713791   0.701983   0.5625                                                                             

[array([0.76589]), 0.562]

In [13]:
# Now try to predict on cats
pred_cats = predict(learn, valid_cats)
print(sum(pred_cats))

237
